In [ ]:
# ============================================================
#  Step 0/8 : Colab Environment Setup
# ============================================================
# ⚠️  This block is intended for Google Colab development only.
#     When running as CLI (grader.py), this section will be skipped.
print("🔹 Step 0/8: Installing minimal dependencies and setting up environment...")

# ------------------------------------------------------------
# 0.1 Mount Google Drive
# ------------------------------------------------------------
from google.colab import drive

try:
    drive.mount("/content/drive")
    print("✅ Google Drive mounted successfully.")
except Exception as e:
    print(f"⚠️ Drive mount skipped or failed: {e}")

# ------------------------------------------------------------
# 0.2 Dependency check & install
# ------------------------------------------------------------
print("📦 Checking required libraries...")
%pip install -q nbformat==5.10.4 pandas==2.2.2

# ------------------------------------------------------------
# 0.3 Suppress warnings (Deprecation, Future)
# ------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
print("🔇 Deprecation and Future warnings are suppressed.")

# ------------------------------------------------------------
# 0.4 Set working directory
# ------------------------------------------------------------
import os, sys
PKG_PARENT = "/content/drive/MyDrive/Colab Notebooks/autograder/src"

try:
    os.chdir(PKG_PARENT)
    sys.path.append(PKG_PARENT)
    print(f"📁 Working directory set to: {PKG_PARENT}")
except Exception as e:
    print(f"⚠️ Directory change failed: {e}")

🔹 Step 0/8: Installing minimal dependencies and setting up environment...
Mounted at /content/drive
✅ Google Drive mounted successfully.
📦 Checking required libraries...
🔇 Deprecation and Future warnings are suppressed.
📁 Working directory set to: /content/drive/MyDrive/Colab Notebooks/autograder/src


In [ ]:
import os
import re
import csv
from pathlib import Path

import nbformat

from datetime import datetime


# ---------------------------------------------------------
# 공통 유틸: 학번/이름 추출
#   - 파일명에서: ..._학번_이름.ipynb 형태 가정
#   - 필요하면 여기 정규식만 살짝 고치면 됨
# ---------------------------------------------------------
def get_student_id_name(path: Path, nb=None):
    filename = path.name

    # 1) 가장 흔한 패턴: ..._학번_이름.ipynb  (학번: 8~10자리)
    m = re.search(r"_(\d{8,10})_([^\.]+)\.ipynb$", filename)
    if m:
        return m.group(1), m.group(2)

    # 2) 예비 패턴: 학번-이름.ipynb, 학번이름.ipynb 등 (혹시 모를 예외 대응)
    m = re.search(r"(\d{8,10})[_-]([^\.]+)\.ipynb$", filename)
    if m:
        return m.group(1), m.group(2)

    # 3) 파일 안에서 찾기 (작년/다른 과제 포맷 등 예외 대비)
    if nb is not None:
        for cell in nb.cells[:5]:
            src = "".join(cell.get("source", ""))

            # 8~10자리 숫자 아무거나 → 학번 후보
            m_id = re.search(r"(\d{8,10})", src)

            # "이름: 홍길동", "성명 = 홍길동" 등
            m_name = re.search(
                r"(?:이름|성명)\s*[:=]\s*['\"]?([가-힣A-Za-z ]+)",
                src
            )

            student_id = m_id.group(1) if m_id else ""
            student_name = m_name.group(1).strip() if m_name else ""
            if student_id or student_name:
                return student_id, student_name

    # 4) 그래도 실패하면 student_name에 일단 파일명을 넣어두기
    return "", filename


def normalize_source(text: str) -> str:
    """비교용 정규화: 양끝 공백 제거 + 각 줄 끝 공백 제거."""
    lines = text.strip().splitlines()
    return "\n".join(line.rstrip() for line in lines)

def get_file_meta(path: Path):
    st = path.stat()
    size_bytes = st.st_size
    mtime_str = datetime.fromtimestamp(st.st_mtime).strftime("%Y-%m-%d %H:%M:%S")
    return mtime_str, size_bytes


# ---------------------------------------------------------
# 11차시: "마지막 셀에 텍스트로 뭔가 적힌" 노트북만 추출
# ---------------------------------------------------------
def collect_11_last_text_cells(submit_dir: Path, output_csv: Path):
    rows = []

    for path in sorted(submit_dir.glob("*.ipynb")):
        try:
            nb = nbformat.read(path, as_version=4)
        except Exception as e:
            print(f"[11차시] {path.name} 읽기 오류: {e}")
            continue

        if not nb.cells:
            continue

        last_cell = nb.cells[-1]
        cell_type = last_cell.get("cell_type", "")
        src = "".join(last_cell.get("source", "")).strip()

        if cell_type in ("markdown", "raw") and src:
            student_id, student_name = get_student_id_name(path, nb)
            file_mtime, file_size = get_file_meta(path)

            rows.append({
                "student_id": student_id,
                "student_name": student_name,
                "file_mtime": file_mtime,
                "file_size_bytes": file_size,
                "last_cell_text": src,
            })

    rows.sort(key=lambda r: r["file_mtime"], reverse=True)

    # CSV 저장 – 헤더에 새 컬럼 추가
    with output_csv.open("w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(
            f,
            fieldnames=[
                "student_id",
                "student_name",
                "file_mtime",
                "file_size_bytes",
                "last_cell_text",
            ],
        )
        writer.writeheader()
        for row in rows:
            writer.writerow(row)

    print(f"[11차시] 추출된 노트북 수: {len(rows)}  → {output_csv}")


# ---------------------------------------------------------
# 12차시: "원본 마지막 텍스트셀과 다른" 노트북만 추출
#   - 원본 과제 노트북(교수님 템플릿)의 마지막 셀을 기준으로 비교
# ---------------------------------------------------------
def collect_12_modified_last_text_cells(
    submit_dir: Path,
    template_nb_path: Path,
    output_csv: Path,
):
    try:
        tmpl_nb = nbformat.read(template_nb_path, as_version=4)
    except Exception as e:
        print(f"[12차시] 템플릿 노트북 읽기 오류: {e}")
        return

    if not tmpl_nb.cells:
        print("[12차시] 템플릿 노트북에 셀이 없습니다.")
        return

    template_last_cell = tmpl_nb.cells[-1]
    template_src_norm = normalize_source("".join(template_last_cell.get("source", "")))

    rows = []

    for path in sorted(submit_dir.glob("*.ipynb")):
        try:
            nb = nbformat.read(path, as_version=4)
        except Exception as e:
            print(f"[12차시] {path.name} 읽기 오류: {e}")
            continue

        if not nb.cells:
            continue

        last_cell = nb.cells[-1]
        src = "".join(last_cell.get("source", ""))
        src_norm = normalize_source(src)

        if src_norm != template_src_norm:
            student_id, student_name = get_student_id_name(path, nb)
            file_mtime, file_size = get_file_meta(path)

            rows.append({
                "student_id": student_id,
                "student_name": student_name,
                "file_mtime": file_mtime,
                "file_size_bytes": file_size,
                "last_cell_text": src.strip(),
            })


    rows.sort(key=lambda r: r["file_mtime"], reverse=True)
    with output_csv.open("w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(
            f,
            fieldnames=[
                "student_id",
                "student_name",
                "file_mtime",
                "file_size_bytes",
                "last_cell_text",
            ],
        )
        writer.writeheader()
        for row in rows:
            writer.writerow(row)

    print(f"[12차시] 추출된 노트북 수: {len(rows)}  → {output_csv}")


# ---------------------------------------------------------
# 실행부: 여기 경로만 교수님 환경에 맞게 바꿔서 사용
# ---------------------------------------------------------

# 👉 이 부분만 교수님 구글드라이브 / 로컬 경로에 맞게 수정하시면 됩니다.
base = Path("/content/drive/MyDrive/Colab Notebooks/52.KHCU_MR25")

# 11차시 제출 폴더 + 결과 CSV
submit_dir_11 = base / "11차시"   # 예시
output_11 = base / "Autograde/mr11_last_textcell.csv"

# 12차시 제출 폴더 + 템플릿 노트북 + 결과 CSV
submit_dir_12 = base / "12차시"   # 예시
template_12 = base / "MR_12_회귀분석_학번_성명.ipynb"  # 학생에게 배포했던 원본 과제 노트북
output_12 = base / "Autograde/mr12_modified_last_textcell.csv"

# 필요에 따라 둘 중 하나만 호출해도 됨
if submit_dir_11.exists():
    collect_11_last_text_cells(submit_dir_11, output_11)
else:
    print(f"[11차시] 제출 폴더가 없습니다: {submit_dir_11}")

if submit_dir_12.exists() and template_12.exists():
    collect_12_modified_last_text_cells(submit_dir_12, template_12, output_12)
else:
    print(f"[12차시] 제출 폴더 또는 템플릿이 없습니다: {submit_dir_12}, {template_12}")

[11차시] 추출된 노트북 수: 20  → /content/drive/MyDrive/Colab Notebooks/52.KHCU_MR25/Autograde/mr11_last_textcell.csv
[12차시] 추출된 노트북 수: 13  → /content/drive/MyDrive/Colab Notebooks/52.KHCU_MR25/Autograde/mr12_modified_last_textcell.csv
